# Практические задания к лекции № 2

In [26]:
import pandas as pd
import numpy as np

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
from gensim.corpora.dictionary import Dictionary

In [4]:
#предобработка текстов
import re
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
import pymorphy2

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pvs-v\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

In [9]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [10]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [11]:
%%time
#Запускаем очистку текста. 
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-10-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 16.5 s


In [12]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 22s


In [13]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [14]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [15]:
common_dictionary[10]

'ватутин'

In [16]:
from gensim.models import LdaModel

In [17]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary, passes=3)

Wall time: 1min 1s


In [18]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [19]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.26950175),
 (3, 0.018455738),
 (4, 0.061357517),
 (7, 0.163757),
 (9, 0.033353053),
 (10, 0.18925849),
 (13, 0.21837434),
 (15, 0.031118033)]

In [20]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: испания команда матч адмирал победа чемпионат арбитраж
topic_1: остров армия экипаж километр воздух сигнал флот
topic_2: год млн цена рост составить тыс рынок
topic_3: век сократиться еда италия образоваться датчик пить
topic_4: украина украинский который закон гражданин год право
topic_5: депутат совет выплата доход зарплата пост президент
topic_6: земля который год время это станция первый
topic_7: россия российский млрд москва проект также nn
topic_8: египет японский корея япония южный курение актёр
topic_9: фотография лёд рисунок дед кит просмотр прага
topic_10: год это который человек мочь весь всё
topic_11: сша американский журнал китай китайский опубликовать иран
topic_12: статья день это человек который всё газета
topic_13: женщина обнаружить свой жизнь смерть который мужчина
topic_14: военный ракета сила операция советский территория боевой
topic_15: форум бизнесмен белый медицина израиль лауреат израильский
topic_16: год суд налог место млн nn рейтинг
topic_17: товар

In [21]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [22]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.030858,0.0,0.000000,0.000000,0.000000,0.020795,0.000000,0.585876,0.0,...,0.013306,0.000000,0.0,0.000000,0.0,0.00000,0.100472,0.000000,0.0,0.241952
1,4896,0.331076,0.0,0.238438,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.061268,0.0,0.000000,0.0,0.00000,0.000000,0.052532,0.0,0.000000
2,4897,0.269532,0.0,0.000000,0.018448,0.061263,0.000000,0.000000,0.163845,0.0,...,0.031117,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000
3,4898,0.127632,0.0,0.000000,0.000000,0.000000,0.000000,0.027717,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.17769,0.162184,0.000000,0.0,0.181681
4,4899,0.000000,0.0,0.291623,0.000000,0.067208,0.433246,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.064401,0.0,0.00000,0.121271,0.000000,0.0,0.000000


### Следующий шаг - векторные представления пользователей

In [28]:
users = pd.read_csv("users_articles.csv")
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [29]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [31]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.00000,0.000000,0.021200,0.002616,0.151779,0.009287,0.040571,0.107242,0.017113,...,0.007286,0.012240,0.011932,0.000000,0.033149,0.103190,0.222959,0.012822,0.035348,0.014995
1,u108690,0.00000,0.002464,0.032520,0.008659,0.122238,0.028704,0.017315,0.026767,0.000000,...,0.002353,0.005680,0.000000,0.015792,0.065533,0.254003,0.042296,0.000000,0.082919,0.057962
2,u108339,0.00399,0.007811,0.031273,0.002248,0.056156,0.006662,0.055660,0.099618,0.000000,...,0.000000,0.011774,0.000000,0.002959,0.162878,0.145582,0.075016,0.000000,0.117099,0.061308


In [33]:
target = pd.read_csv("users_churn.csv")
X = pd.merge(user_embeddings, target, 'left')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [36]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [38]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.366352, F-Score=0.822, Precision=0.902, Recall=0.755


In [39]:
scores_mean = [roc_auc_score(y_test, preds), fscore[ix], precision[ix], recall[ix]]
scores_mean

[0.9797174254317111,
 0.8222222222222222,
 0.9024390243902439,
 0.7551020408163265]

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [40]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

In [41]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [42]:
X = pd.merge(user_embeddings, target, 'left')
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [43]:
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [44]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.328657, F-Score=0.837, Precision=0.858, Recall=0.816


In [45]:
scores_median = [roc_auc_score(y_test, preds), fscore[ix], precision[ix], recall[ix]]
scores_median

[0.98438048723763, 0.8368200836820083, 0.8583690987124464, 0.8163265306122449]

3. Повторить п.2, но используя уже не медиану, а max

In [46]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [47]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [48]:
X = pd.merge(user_embeddings, target, 'left')
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [49]:
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [50]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.407572, F-Score=0.812, Precision=0.824, Recall=0.800


In [51]:
scores_max = [roc_auc_score(y_test, preds), fscore[ix], precision[ix], recall[ix]]
scores_max

[0.978912727484156, 0.8115942028985507, 0.8235294117647058, 0.8]

4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [54]:
df = pd.DataFrame([scores_mean, scores_median, scores_max], 
                  columns = ['roc_auc_score', 'fscore', 'precision', 'recall'], 
                  index = ['mean', 'median', 'max'])
df

,roc_auc_score,fscore,precision,recall
mean,0.979717,0.822222,0.902439,0.755102
median,0.984380,0.836820,0.858369,0.816327
max,0.978913,0.811594,0.823529,0.800000


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных